In [22]:
# pip install langchain
# pip install chromadb
# pip install pypdf

In [23]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.vectorstores import Chroma

In [24]:
pdf_loader=PyPDFDirectoryLoader("pdfs")

In [25]:
data=pdf_loader.load()

In [ ]:
data[0].page_content.strip()

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [28]:
text_chunks=text_splitter.split_documents(data)

In [ ]:
text_chunks[0]

### Creating DB

In [30]:
persist_directory="db"

In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [33]:
vectordb=Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
vectordb.persist()

In [35]:
vectordb=None

In [36]:
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)

In [ ]:
vectordb

### getting the DB

In [38]:
retriver=vectordb.as_retriever()

In [ ]:
docs=retriver.get_relevant_documents("Who Is anurag singh and where from he?")

In [ ]:
docs

In [41]:
import os
from dotenv import load_dotenv

load_dotenv()
google_gemini_api=os.getenv("GOOGLE_API_KEY")

In [ ]:
llm_model=ChatGoogleGenerativeAI(model="gemini-1.5-pro",google_api_key=google_gemini_api)

In [43]:
system_prompt = (
    "You are an expert in question and answering. Given the following context, it is your job to provide a useful and concise answer. Make sure to ask a follow-up question in a dramatic and funny way at the end to make the interaction engaging and a bit humorous,funny and naughty, but only where it seems appropriate."
    "Use emojies"
    "Context: {context}"
)

In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm_model, prompt)
chain = create_retrieval_chain(retriver, question_answer_chain)

In [ ]:
chain

In [47]:
query="is anurag singh have any best project?"

In [ ]:
chain.invoke({"input": query})

In [ ]:
pdf_loaderr=PyPDFDirectoryLoader("../../media/pdfs")
documentsss = pdf_loaderr.load()
documentsss